<a href="https://colab.research.google.com/github/opencoca/vidsum/blob/master/GPT_J_6B_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-J-6B Inference 

This notebook explores how to work with the [GPT-J-6B model](https://github.com/kingoflolz/mesh-transformer-jax/#GPT-J-6B). See the link for more details about the model, including evaluation metrics and credits.

Please note it takes ~10 minutes for this notebook to fully spin up. Sometimes you will runout of ram when attempting to setup the model. If this happens factory restart the runtime and try again. If it contines to happen ...

Start this notebok by clicking `Runtime > Run all` or clicking `⌘/Ctrl+F9`. Use `⌘` on Mac and `Ctrl` on Windows and Linux. 

Keep an eye on this as it spins up. Google likes to make sure you are not a bot and will stop things if you ignore the "I'm not a robot" popup.

## Prepare it to be used

### Install Dependencies

First we download the model and install some dependencies. This step takes at least 5 minutes (possibly longer depending on server load).

!!! **Make sure you are using a TPU runtime!** !!!

In [ ]:
#@title Install Tensorflow
!pip install tensorflow==2.5.0

In [ ]:
#@title Install GPT-J-6B
!apt install zstd

# the "slim" version contain only bf16 weights and no optimizer parameters, which minimizes bandwidth and memory
!time wget -c https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd

!time tar -I zstd -xf step_383500_slim.tar.zstd

!git clone https://github.com/kingoflolz/mesh-transformer-jax.git
!pip install -r mesh-transformer-jax/requirements.txt

# jax 0.2.12 is required due to a regression with xmap in 0.2.13
!pip install mesh-transformer-jax/ jax==0.2.12

## Setup Model

You may have to restart your runtime before this will work. 



In [ ]:
#@title Setup JAX for TPU use
import os
import requests 
from jax.config import config

colab_tpu_addr = os.environ['COLAB_TPU_ADDR'].split(':')[0]
url = f'http://{colab_tpu_addr}:8475/requestversion/tpu_driver0.1_dev20210607'
requests.post(url)

# The following is required to use TPU Driver as JAX's backend.
config.FLAGS.jax_xla_backend = "tpu_driver"
config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']

Sometimes the next step errors for some reason, just run it again ¯\\\_(ツ)\_/¯

In [ ]:
#@title Import libraries
import time

import jax
from jax.experimental import maps
import numpy as np
import optax
import transformers

from mesh_transformer.checkpoint import read_ckpt
from mesh_transformer.sampling import nucleaus_sample
from mesh_transformer.transformer_shard import CausalTransformer

In [ ]:
#@title Set Network Parameters 
params = {
  "layers": 28,
  "d_model": 4096,
  "n_heads": 16,
  "n_vocab": 50400,
  "norm": "layernorm",
  "pe": "rotary",
  "pe_rotary_dims": 64,

  "seq": 2048,
  "cores_per_replica": 8,
  "per_replica_batch": 1,
}

per_replica_batch = params["per_replica_batch"]
cores_per_replica = params["cores_per_replica"]
seq = params["seq"]


params["sampler"] = nucleaus_sample

# here we "remove" the optimizer parameters from the model (as we don't need them for inference)
params["optimizer"] = optax.scale(0)

mesh_shape = (jax.device_count() // cores_per_replica, cores_per_replica)
devices = np.array(jax.devices()).reshape(mesh_shape)

maps.thread_resources.env = maps.ResourceEnv(maps.Mesh(devices, ('dp', 'mp')))

tokenizer = transformers.GPT2TokenizerFast.from_pretrained('gpt2')

In [ ]:
#@title Install Tracery 
#@markdown Tracery is a gramar for spining articles, stories, and text in general. https://github.com/aparrish/pytracery
!pip install tracery

In [ ]:
#@title Tidy things up
from IPython.display import clear_output
clear_output()

In [7]:
#@title Create the network
#@markdown ...loading neural weights from the downloaded files. 
#@markdown
#@markdown  > *This can take around 5 minutes.*
total_batch = per_replica_batch * jax.device_count() // cores_per_replica

network = CausalTransformer(params)

network.state = read_ckpt(network.state, "step_383500/", devices.shape[1])

network.state = network.move_xmap(network.state, np.zeros(cores_per_replica))

Total parameters: 6053381344
read from disk/gcs in 34.895s


# Run Model

Finally, we are ready to infer with the model! The first sample takes around a minute due to compilation, but after that it should only take about 10 seconds per sample.

Feel free to mess with the different sampling parameters (top_p and temp), as well as the length of the generations (gen_len, causes a recompile when initially changed - recompiles will be cached).

You can also change other things like per_replica_batch in the previous cells to change how many generations are done in parallel. A larger batch has higher latency but higher throughput when measured in tokens generated/s. This is useful for doing things like best-of-n cherry picking.

*Tip for best results: Make sure your prompt does not have any trailing spaces, which tend to confuse the model due to the BPE tokenization used during training.*

In [8]:
#@markdown We allow for text wrapping of whitespace via css injection.

# allow text wrapping in generated output: https://stackoverflow.com/a/61401455
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [9]:
#@title Define our infer function. 
def infer(context, top_p=0.9, temp=1.0, gen_len=256):
    tokens = tokenizer.encode(context)

    provided_ctx = len(tokens)
    pad_amount = seq - provided_ctx

    padded_tokens = np.pad(tokens, ((pad_amount, 0),)).astype(np.uint32)
    batched_tokens = np.array([padded_tokens] * total_batch)
    length = np.ones(total_batch, dtype=np.uint32) * len(tokens)

    start = time.time()
    output = network.generate(batched_tokens, length, gen_len, {"top_p": np.ones(total_batch) * top_p, "temp": np.ones(total_batch) * temp})

    samples = []
    decoded_tokens = output[1][0]

    for o in decoded_tokens[:, :, 0]:
      samples.append(f"\033[1m{context}\033[0m{tokenizer.decode(o)}")

    #print(f"completion done in {time.time() - start:06}s")
    #this should instead be added to the object returned infered.samples and infered.completion_time
    return samples




In [10]:
#@title Warmup our inference engine 
#@markdown The first time that infrences are run for a given length takes imensly longer than on repeated runs. For this reason we spin up the inference engine with a few sample runs. With this out of the way we can quickly experiment with our infrence engine. 

context = """# A NEW WAY TO WORK IN THE OLD PORT OF MONTREAL AT SPACES CITÉ MULTIMÉDIA

## The Old Port of Montreal is a"""

for i in [32,64,128,256,512,1024]:
  print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
  print(f" ## Infering context with a gen_len of {i} this should take ~ {i/16} sec. when run again ##")
  print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
  print(infer(context, gen_len=i)[0])

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
 ## Infering context with a gen_len of 32 this should take ~ 2.0 sec. when run again ##
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# A NEW WAY TO WORK IN THE OLD PORT OF MONTREAL AT SPACES CITÉ MULTIMÉDIA

## The Old Port of Montreal is a large urban park for all to enjoy, offering more than four hundred _places_ for leisure, the arts, sports, study, and relaxation, along with numerous
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
 ## Infering context with a gen_len of 64 this should take ~ 4.0 sec. when run again ##
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# A NEW WAY TO WORK IN THE OLD PORT OF MONTREAL AT SPACES CITÉ MULTIMÉDIA

## The Old Port of Montreal is a Field of Opportunity

Barcelona has been a city of international distinction for decades. However,

# Example prompts

We've compiled the model. At this point it should only take about 15 seconds per sample. The first time you run inference on a new dataset takes longer than usual because of compilation to build the graph and write data into disk buffers for fast access by CPUs/GPUs.

The following parameters can be adjusted: top_p, temp, gen_len. When changing the length of generations (gen_len), recompilation is required to maintain accuracy and consistency in results.

You can also change other things like the number of generations done in parallel and how many samples are generated. In addition, you can increase or decrease the batch size for latency/throughput tradeoffs as well. This is useful when performing best-of-n cherry picking tasks where throughput needs to be maximized with minimum batch time delay on each sample generation process.

To get the best results, make sure your prompt does not have any trailing spaces. This is because BPE tokenization can be confused by them.

## Simple Inference 

### Set Sampling Parameters


In [11]:
#@markdown ### Leave this alone unless you know what you are doing ... or are just having fun experimenting.
#@markdown 
#@markdown - - - 
#@markdown Top_p controls diversity/quality. The higher our top_p the more rich and nuanced oir inferences can be.
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Temp controls randomnes. Low temps make for mechanical inferences.
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Gen_len adjects the length of text infered.
#@markdown Specifcily it adustes the nember of tokens (~4 per engish word).
#@markdown
#@markdown *Note: Changing gen_len will cause long wait times when first setting lengs that have not been set before.*
#@markdown  
gen_len = 64 #@param {type: "slider", min:64, max: 1024, step: 64}
#@markdown - - - 
#@markdown Generate controls the number of inferences that will be generated.
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}


### Text examples

In [38]:
#@title Raw Context 
#@markdown This is an example of simple summery infrence.

#print("Enter/Paste your content. Ctrl-D or Ctrl-Z ( windows ) to save it.")
#context = []
#while True:
#    try:
#        line = input()
#    except EOFError:
#        break
#    context.append(line)

context = """"

Article:

t’s an intense crypto dogfight as dogecoin (DOGE) and shiba inu (SHIB), two of the most popular meme coins, battle it out for the ninth spot on the list of top digital assets by market capitalization. Some traders are profiting from the action by taking spread trades.

DOGE has come alive after lagging SHIB by a significant margin earlier this month. The joke cryptocurrency surged to $0.335 on Coinbase on Thursday, hitting the highest level since Aug. 20. It was last trading near $0.30, representing a 22% gain on the day.

Meanwhile, SHIB crashed as much as 30% to $0.00006 earlier today, having chalked out a rally to $0.00009 in the seven days to Oct. 27.

Summery:"""

print(infer(top_p=top_p, temp=1, gen_len=64, context=context)[0])

"

Article:

t’s an intense crypto dogfight as dogecoin (DOGE) and shiba inu (SHIB), two of the most popular meme coins, battle it out for the ninth spot on the list of top digital assets by market capitalization. Some traders are profiting from the action by taking spread trades.

DOGE has come alive after lagging SHIB by a significant margin earlier this month. The joke cryptocurrency surged to $0.335 on Coinbase on Thursday, hitting the highest level since Aug. 20. It was last trading near $0.30, representing a 22% gain on the day.

Meanwhile, SHIB crashed as much as 30% to $0.00006 earlier today, having chalked out a rally to $0.00009 in the seven days to Oct. 27.

Summery:

Both DOGE and SHIB have registered consecutive gains over the past week. DOGE has been on an uptrend and is expected to gain further momentum in the near term. However, the sentiment has started to weaken and the coin is at risk of testing the bearish trendline. As SHIB


In [13]:
#@title Expand into a script 
#@markdown The will expand a lead into a script bassed on the opening of pulp fiction.
write_script_on_this = "Feline biology is amazing" #@param {type:"string"}

#@markdown - - - - - - - - - - - - - - - - - - - - - - - - -
#@markdown These options don't need to be adjusted.
top_p = 0.9 #@ param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@ param {type:"slider", min:0, max:1, step:0.1}
gen_len = 512 #@param {type: "slider", min:64, max: 2048, step: 64}
generate = 1 #@ param {type: "slider", min:1, max: 12, step: 1}
#@markdown - - - - - - - - - - - - - - - - - - - - - - - - -


context = """

INT. COMPUTER LAB – MORNING

A normal computer lab, an office or university, Toronto Canada.
It's about 9:00 in the morning. While the place isn't jammed,
there's a healthy number of people working, drinking coffee, 
reading at their computers.

Two of these people are a YOUNG MAN and a YOUNG WOMAN. The
Young Man has a slight working-class English accent.

             YOUNG MAN
  I want to tell you about what we've been studying.

             YOUNG WOMAN
  Oh? What have you been focusing on? 

             YOUNG MAN
  {expand_this}. For example"""


for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=context.format(expand_this = expand_this))[0])
  except:
    pass



INT. COMPUTER LAB – MORNING

A normal computer lab, an office or university, Toronto Canada.
It's about 9:00 in the morning. While the place isn't jammed,
there's a healthy number of people working, drinking coffee, 
reading at their computers.

Two of these people are a YOUNG MAN and a YOUNG WOMAN. The
Young Man has a slight working-class English accent.

             YOUNG MAN
  I want to tell you about what we've been studying.

             YOUNG WOMAN
  Oh? What have you been focusing on? 

             YOUNG MAN
  Feline biology is amazing. For example, they can talk.

             YOUNG WOMAN
  You're making that up.

             YOUNG MAN
  I've been doing research.
                            
  We've been studying more about felines, though.

The two of them chat and their eyes meet, and there is a
moment of connection. They smile at each other. They're
friends. Then they get back to work.

INT. TERRACE – DAY

             YOUNG MAN
  I'm looking for a good biologist.

   

In [14]:
#@title Make a title sound more academic 
#@markdown  
improve_this = "we're going ti make an instrument using the potentiometer " #@param {type:"string"}
#@markdown - - -
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 64 #@param {type: "slider", min:64, max: 2048, step: 64}
generate = 12 #@param {type: "slider", min:1, max: 12, step: 1}

context = """
Instead of saying, "{improve_this}" a better shibboleth would be, \""""

#Indie Hackers want to click on titles such as 
#use something with more punch

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=context.format(improve_this = improve_this))[0].split('"')[3])
  except:
    pass

we're going to make an instrument that does something.
we're going to make an instrument that is made out of a potentiometer 
we're going to use a potentiometer.
we're going ti make an instrument using the potentiometer 
we're going to use the MOSFET
we're going ti find the value of R2 when the pot is fully open
we're going to manufacture one.
we're going ti ask a potiometer to make an instrument
we're going to make an instrument that uses the potentiometer.
we're going ti make a source of feedback using a potentiometer 
we're going to use the potentiometer 
what are you going to do with the variable resistor?


In [15]:
#@title Give headlines some more Clickbait 
improve_this = "Make an instrument using the a light sensor " #@param {type:"string"}
#@markdown - - - - - - - - - - - - - - - - - - - - - - - - 
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 1 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 64 #@param {type: "slider", min:64, max: 2048, step: 64}
generate = 12 #@param {type: "slider", min:1, max: 12, step: 1}

context = """
You need to make your title more clickbate. So instead of,
"{improve_this}" use something with more punch such as,\""""


for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=context.format(improve_this = improve_this))[0].split('"')[3])
                #context=context.format(improve_this = improve_this))[0])

  except:
    pass

 
 


 
 
 turn a light bulb into a vibrator
 How to make a light sensor guitar
Make a clicker using an ambient light sensor
 Make your own bazooka. 
 
 
 how can you use a light sensor to make a heart beat.


In [16]:
#@title Generating inference prompts using tracery
import tracery
from tracery.modifiers import base_english

rules = {
    
    'origin': 'Our team builds #imagine# new online #location# beyond',
    'imagine': ['amazing', 'spectaculare' , 'bold', ],
    'location': ['worlds', 'spaces', 'colabortion']
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=grammar.flatten("#origin#"))[0] + "\n")
  except:
    pass

Our team builds bold new online worlds beyond what you see today and is led by co-founders Eric Widera and Daniel Whiteson. We've previously launched award-winning video games such as Machinarium and Braid, and this summer, the world premiere of our hit game Monument Valley.

Company: Big Fish Games

Big Fish

Our team builds amazing new online worlds beyond Second Life that are accessible on any device. This includes mobile, desktop and even, web. We work hard to build for all of these platforms, and are pleased to announce that we are making our initial foray into HTML5 just in time for WorldMaker’s fifth birthday!

The beginnings

Our

Our team builds amazing new online spaces beyond Facebook. Please give us a shout if you’d like to speak with us about the sites we built and support, or learn more about realtime streamers and what makes them tick.

Video Transcription

Becca and I, we are here to bring you a video! Becca is our

Our team builds spectaculare new online worlds beyond 

In [17]:
#@title Generating inference prompts using tracery
import tracery
from tracery.modifiers import base_english

rules = {
    
    'origin': '#this_semester# Jane #does# #good_bad# in class. We hope that she',
    'this_semester': ['Our focus this semester was learning to program microcomputers, using sensors, and building electronic circuits.','This semester we focused on learning to build electric circuits, reading information from sensors, and programming microcontrollers.'],
    'does': ['works', 'does' , 'particpates', 'performs' ],
    'good_bad': ['well', 'competently', 'baddly']
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)

for x in range(generate):
  try:
    print(infer(top_p=top_p, 
                temp=temp, 
                gen_len=gen_len, 
                context=grammar.flatten("#origin#"))[0] + "\n")
  except:
    pass

This semester we focused on learning to build electric circuits, reading information from sensors, and programming microcontrollers. Jane particpates well in class. We hope that she can continue to grow in her learning by using Arduino as a medium of both expression and problem solving. As we progress, she will learn that not all technical challenges can be addressed by a microcontroller. In the past, her projects have used very sophisticated electronics as well as electronic controls. This semester we hope to move beyond

This semester we focused on learning to build electric circuits, reading information from sensors, and programming microcontrollers. Jane does competently in class. We hope that she will apply what she learns with enthusiasm in her future career endeavors. Good luck, Jane! —Carmen

Vilas’ students were fortunate to have had Professor Paul Kuhn speak to the class. Kuhn spoke about his home life in Oklahoma City, his early education at the University of Oklahoma and

O

In [18]:
#@title Article Generator 
title = "Being your own Boss" #@param {type:"string"}
site = "" #@param {type:"string"}
date = "" #@param {type:"string"}
author = "Jim Rohn" #@param {type:"string"}
tags = "goals, drive, gtd" #@param {type:"string"}
lead = "## Table of Content" #@param {type:"string"}
#@markdown - - -

context = """
# {title}
### Published:{date}
### By:{author}
##### Tags: {tags}
### Site: {site}

{lead}
"""

for x in range(generate):
  print(infer(top_p=top_p, 
              temp=temp, 
              gen_len=gen_len, 
              context=context.format(title =title, 
                                     site = site, 
                                     date = date, 
                                     author = author, 
                                     tags = tags, 
                                     lead = lead))[0])


# Being your own Boss
### Published:
### By:Jim Rohn
##### Tags: goals, drive, gtd
### Site: 

## Table of Content

##  Getting Started  
### The Goal

In business, you're your own boss. You have the freedom to make your own decisions, use your own time and choose your own projects and goals. With _Own The Goal_, you will learn how to make smart, strategic decisions, see yourself

# Being your own Boss
### Published:
### By:Jim Rohn
##### Tags: goals, drive, gtd
### Site: 

## Table of Content

### Introduction

### The Beginning

### Checklist

#### Introduction

To understand the psychology of doing, you have to start at the very beginning. I want you to think of yourself as being behind the wheel of a car in your mind's eye. You are getting ready to set off on

# Being your own Boss
### Published:
### By:Jim Rohn
##### Tags: goals, drive, gtd
### Site: 

## Table of Content

### Being your own Boss

> You must  (careful, it  is easy to say) start with   as a young man,
>  finish wi

In [19]:
#@title Article generation { form-width: "300px" }
title = "COVID-19: What do vaccine passports in Quebec & Ontario mean for Canada's largest provinces? " #@param {type:"string"}
site = "https://www.canada.ca/en/public-health/services/diseases/" #@param {type:"string"}
date = "Sep 7, 2021" #@param {type:"string"}
author = "" #@param {type:"string"}
tags = "COVID-19, Quebec, Ontario, Canada" #@param {type:"string"}
lead = "The passports are certificates that confirm vaccinations and allow people to enjoy eating" #@param {type:"string"}


#print("Enter/Paste your content. Ctrl-D or Ctrl-Z ( windows ) to save it.")
#context = []
#while True:
#    try:
#        line = input()
#    except EOFError:
#        break
#    context.append(line)

context = """
# {title}
### Published:{date}
### By:{author}
##### Tags: {tags}
### Site: {site}

{lead}"""

for x in range(generate):
  print(infer(top_p=top_p, 
              temp=temp, 
              gen_len=gen_len, 
              context=context.format(title =title, 
                                     site = site, 
                                     date = date, 
                                     author = author, 
                                     tags = tags, 
                                     lead = lead))[0])


# COVID-19: What do vaccine passports in Quebec & Ontario mean for Canada's largest provinces? 
### Published:Sep 7, 2021
### By:
##### Tags: COVID-19, Quebec, Ontario, Canada
### Site: https://www.canada.ca/en/public-health/services/diseases/

The passports are certificates that confirm vaccinations and allow people to enjoy eating fish, honey, and maple syrup from Quebec and Ontario.

_Euronews_: "Alberta is the only other province with a validated emissions-reduction plan, though it's since been scrapped."

In response, Manitoba Provincial Health Officer Theresa Oswald stated that the province had not made its

# COVID-19: What do vaccine passports in Quebec & Ontario mean for Canada's largest provinces? 
### Published:Sep 7, 2021
### By:
##### Tags: COVID-19, Quebec, Ontario, Canada
### Site: https://www.canada.ca/en/public-health/services/diseases/

The passports are certificates that confirm vaccinations and allow people to enjoy eating raw or undercooked fish and seafood.

Acco

## Code Generation

### Set Sampling Parameters


In [20]:
#@markdown ### Leave this alone unless you know what you are doing ... or are just having fun experimenting.
#@markdown 
#@markdown - - - 
#@markdown Top_p controls diversity/quality. The higher our top_p the more rich and nuanced oir inferences can be.
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Temp controls randomnes. Low temps make for mechanical inferences.
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown - - - 
#@markdown Gen_len adjects the length of text infered.
#@markdown Specifcily it adustes the nember of tokens (~4 per engish word).
#@markdown
#@markdown *Note: Changing gen_len will cause long wait times when first setting lengs that have not been set before.*
#@markdown  
gen_len = 512 #@param {type: "slider", min:64, max: 1024, step: 64}
#@markdown - - - 
#@markdown Generate controls the number of inferences that will be generated.
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}


### Python Code

In [21]:
#@title Python Code - Simple 
#@markdown Generate code by just writng a comment and a name for a fuction 
python_comment = "check if a number is prime or not" #@param {type:"string"}
#variable_n_val = "youtube_video_url" #@param {type:"string"}
function_name = "p" #@param {type:"string"}


context = """# {python_comment}

def is_""".format(python_comment = python_comment, 
           function_name = function_name)

for x in range(generate):
  print(infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0])

# check if a number is prime or not

def is_prime(num):
    if num == 0:
        return True

    for i in range(2, num):
        if num % i == 0:
            return False
    return True

def prime_list():
    numbers = []

    for x in range(1, 100):
        if is_prime(x):
            numbers.append(x)

    return numbers

print(prime_list())

A:

You can use itertools.combinations
from itertools import combinations

def is_prime(num):
    return all(num % x for x in combinations(range(2, num), 2))

def prime_list():
    numbers = []
    for x in range(1, 100):
        if is_prime(x):
            numbers.append(x)
    return numbers

print(prime_list())

A:

A solution using math, a solution using any and only solution using recursion (it's quite an interesting one) are given.
According to my personal taste, the recursion version is the most elegant.
from math import sqrt

def is_prime(n):
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    if n < 0:
    

In [22]:
#@title Python Code - Complex
python_comment = "detect security flaws " #@param {type:"string"}
variable_name ="person_real_name" #@param {type:"string"}
variable_val = "Isabelle Plante" #@param {type:"string"}
function_name = "online_account_scan" #@param {type:"string"}
#@markdown - - -


context = '''#!/usr/bin/env python3

# {python_comment}

{variable_name} = "{variable_val}"

def {function_name}(n):
  """This'''.format(python_comment = python_comment,
           variable_name = variable_name,
           variable_val = variable_val,
           function_name = function_name)

for x in range(generate):
  print(infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0])

#!/usr/bin/env python3

# detect security flaws 

person_real_name = "Isabelle Plante"

def online_account_scan(n):
  """This is a basic online account scan. 

  It may return more than 1 result for the "account"
  parameter if there are more than one online account configured on the device.

  It is assumed that the account details exist in n. 
  The account details include:
  - phone number
  - password
  - name
  - login time
  - last login time

  Notes:
  - "n" is a list, so you can just add a list of phone numbers, passwords,
    names, etc..
  - This function will check all accounts, but the first entry will be the one
    that is checked first.
  - Most (if not all) of the information may be retrieved via a simple OAuth
    API access.

  @param n: list of account information
  @return: list of account information
  """
  from urllib.request import Request, urlopen
  from io import StringIO
  from json import dumps
  from simplejson import loads
  from urllib import unquote
  i

In [23]:
#@title Vue Code - Simple { form-width: "500px" }
comment = "guest speakers" #@param {type:"string"}
#variable_n_val = "youtube_video_url" #@param {type:"string"}
name = "tab_1" #@param {type:"string"}
#@markdown - - - 
#@markdown Leave this alone unless you know ...
top_p = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
gen_len = 512 #@param {type:"slider", min:64, max:1024, step:64}
generate = 1 #@param {type: "slider", min:1, max: 12, step: 1}

context = """
<!-- Vue template for {comment} page -->
<section>
  <div id="app" class="container">
    <div class="content">
      <div id="{name} " class=""".format(comment = comment, 
           name = name)
      
print(infer(top_p=top_p, temp=temp, gen_len=gen_len, context=context)[0])


<!-- Vue template for guest speakers page -->
<section>
  <div id="app" class="container">
    <div class="content">
      <div id="tab_1 " class= "tabs-panel" >
        <div class="row">
          <div class="col-md-4 col-md-offset-4">
            <div class="panel panel-default">
              <div class="panel-heading">
                <h3 class="panel-title">@{{ title }}</h3>
              </div>
              <div class="panel-body">
                <p>Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum at
                  viverra nunc. Fusce quis metus in dolor semper dignissim ut a odio.
                  Nullam id metus id urna tristique adipiscing. Pellentesque id
                  felis sit amet nisl imperdiet volutpat. Cras quis varius erat.
                  Vivamus elementum porta odio, ac ultrices mauris porttitor sit
                  amet. Duis in augue nunc. Mauris nec sapien eu libero tristique
                  porttitor a a ante. Vivamus elementum 

# Examples of GPT genereated code.

The following are examples of code generated in prior runs.

Please paste interesting examples below

In [24]:
# check if a number is prime or not

def is_prime(n):
  """ 
  > returns true if n is a prime,
  > false otherwise.
  """
  for i in range(2, n):
    if n % i == 0:
      return False
  return True

print(is_prime(7))

True


In [25]:
# Program to check if a number is prime or not

num = 9

# Program to check if a number is prime or not

def is_prime(n):
    while n > 1:
        while n % 2 == 0:
            return False
        n = n / 2
    return True

print("This is an example of a function that works but is quite wrong!")

for i in range(2, 20):
  print(f"Is {i} prime? {is_prime(i)}")



This is an example of a function that works but is quite wrong!
Is 2 prime? False
Is 3 prime? True
Is 4 prime? False
Is 5 prime? True
Is 6 prime? False
Is 7 prime? True
Is 8 prime? False
Is 9 prime? True
Is 10 prime? False
Is 11 prime? True
Is 12 prime? False
Is 13 prime? True
Is 14 prime? False
Is 15 prime? True
Is 16 prime? False
Is 17 prime? True
Is 18 prime? False
Is 19 prime? True


In [26]:
# check if a number is prime or not
def is_prime(n):
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

is_prime(9)

False

In [ ]:
!pip install grip

In [27]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://luw643h0rjp-496ff2e9c6d22116-8000-colab.googleusercontent.com/
